In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [1]:
import dask
import numpy as np
import pandas as pd

In [2]:
from distributed import Client
import distributed as d
print(d.__file__)

/workspace/src/dask/distributed/distributed/__init__.py


In [53]:
client = Client("tcp://scheduler:8786")
client

Client Scheduler: tcp://scheduler:8786 Dashboard: http://scheduler:8787/status,Cluster Workers: 1 Cores: 8 Memory: 33.57 GB


In [4]:
# checking that we have all this stuff in our env
import dask.dataframe as dd
import dask.array as da
from dask_ml.model_selection import train_test_split
import dask
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from scipy.stats import loguniform, uniform
from dask_ml.model_selection import HyperbandSearchCV
from dask_ml.wrappers import ParallelPostFit
from sklearn.metrics import confusion_matrix
import numpy as np
import torch
import time

In [88]:
torch.cuda.is_available()

False

# Small Fake Net

In [6]:
nmb=10000
nfeat=20

In [7]:
features=da.random.normal(size=(nmb,nfeat)).astype("float32")
output=da.random.normal(size=(nmb,)).astype("float32")

In [8]:
from dask_ml.model_selection import train_test_split
import dask

#features = inp.to_dask_array(lengths=True).astype("float32")
#output = out.to_dask_array(lengths=True).astype("float32")
X_train, X_test, y_train, y_test = train_test_split(features, output, shuffle=True)

# persist the data so it's not re-computed
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [9]:
X_train.compute().shape

(9000, 20)

In [10]:
# Import our HiddenLayerNet pytorch model from a local torch_model.py module
from torch_model import HiddenLayerNet
# Send module with HiddenLayerNet to workers on cluster
client.upload_file("torch_model.py")

In [11]:
from skorch import NeuralNetRegressor

niceties = {
    "callbacks": False,
    "warm_start": True,
    "train_split": None,
    "max_epochs": 1,
}

model = NeuralNetRegressor(
    module=HiddenLayerNet,
    module__n_features=X_train.shape[1],
    optimizer=optim.SGD,
    criterion=nn.MSELoss,
    lr=0.0001,
    **niceties,
)

In [12]:
from scipy.stats import loguniform, uniform

params = {
    "module__activation": ["relu", "elu", "softsign", "leaky_relu", "rrelu"],
    "batch_size": [32, 64, 128, 256],
    "optimizer__lr": loguniform(1e-4, 1e-3),
    "optimizer__weight_decay": loguniform(1e-6, 1e-3),
    "optimizer__momentum": uniform(0, 1),
    "optimizer__nesterov": [True],
}

In [13]:
from dask_ml.model_selection import HyperbandSearchCV
search = HyperbandSearchCV(model, params, random_state=42, verbose=True, max_iter=9)

In [14]:
y_train2 = y_train.reshape(-1, 1).persist()
_ = search.fit(X_train, y_train2)

[CV, bracket=2] creating 9 models
0
<Future: pending, key: _create_model-54f12c088d8010ccb3f28dfc55429fd4>
[CV, bracket=1] creating 5 models
0
<Future: pending, key: _create_model-b91f22ae4928f08a134421e51d77f885>
[CV, bracket=0] creating 3 models
0
<Future: pending, key: _create_model-de6407238dc91fd2a294c8c3cd786d22>
(<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class 'torch_model.HiddenLayerNet'>,
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=20, out_features=100, bias=True)
    (fc2): Linear(in_features=100, out_features=1, bias=True)
  ),
  module__activation=leaky_relu,
  module__n_features=20,
), {'model_id': 0, 'params': {'batch_size': 128, 'module__activation': 'leaky_relu', 'optimizer__lr': 0.0008927180304353625, 'optimizer__momentum': 0.7319939418114051, 'optimizer__nesterov': True, 'optimizer__weight_decay': 6.251373574521747e-05}, 'partial_fit_calls': 0})
1
<Future: finished, type: builtins.tuple, key: _create_model-d938b0ecd61c216a57e3

(<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class 'torch_model.HiddenLayerNet'>,
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=20, out_features=100, bias=True)
    (fc2): Linear(in_features=100, out_features=1, bias=True)
  ),
  module__activation=softsign,
  module__n_features=20,
), {'model_id': 6, 'params': {'batch_size': 256, 'module__activation': 'softsign', 'optimizer__lr': 0.000125986196637684, 'optimizer__momentum': 0.5656236363548182, 'optimizer__nesterov': True, 'optimizer__weight_decay': 0.0006714980653732507}, 'partial_fit_calls': 0})
7
<Future: finished, type: builtins.tuple, key: _create_model-85703cf2afe1285f5b759b274741463a>
(<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class 'torch_model.HiddenLayerNet'>,
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=20, out_features=100, bias=True)
    (fc2): Linear(in_features=100, out_features=1, bias=True)
  ),
  module__activation=rrelu,
  module__n_featur

In [16]:
search.best_score_

0.000612503383119134

In [17]:
search.best_params_

{'batch_size': 256,
 'module__activation': 'softsign',
 'optimizer__lr': 0.00010485387725194618,
 'optimizer__momentum': 0.9699098521619943,
 'optimizer__nesterov': True,
 'optimizer__weight_decay': 0.0003142880890840108}

In [18]:
search.best_estimator_

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=20, out_features=100, bias=True)
    (fc2): Linear(in_features=100, out_features=1, bias=True)
  ),
)

# Small Fake Classifier

In [5]:
nmb=10000
nfeat=10
nclasses=5

In [6]:
features=da.random.normal(size=(nmb,nfeat)).astype("float32")

In [7]:
outclasses = np.random.randint(0, nclasses, (nmb,))

In [8]:
d_outclasses=da.from_array(outclasses)

In [11]:
#output = np.zeros((nmb,nclasses)).astype('float32')
#output[(range(nmb),outclasses)] = 1
#output
#output.sum(axis=1)
#output = da.from_array(output)

In [12]:
from dask_ml.model_selection import train_test_split
import dask

#features = inp.to_dask_array(lengths=True).astype("float32")
#output = out.to_dask_array(lengths=True).astype("float32")
X_train, X_test, y_train, y_test = train_test_split(features, d_outclasses, shuffle=True)

# persist the data so it's not re-computed
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [13]:
X_train.compute().shape

(9000, 10)

In [14]:
y_train.compute()

array([4, 1, 3, ..., 0, 0, 1])

In [ ]:
import importlib
import torch_model

importlib.reload(torch_model)

# Import our HiddenLayerNet pytorch model from a local torch_model.py module
from torch_model import HiddenLayerNet
# Send module with HiddenLayerNet to workers on cluster
client.upload_file("torch_model.py")

## Test Local Training

In [143]:
from skorch import NeuralNetClassifier

niceties = {
    "callbacks": False,
    "warm_start": True,
    "train_split": None,
    "max_epochs": 100,
}

model = NeuralNetClassifier(
    module=HiddenLayerNet,
    module__n_features=X_train.shape[1],
    module__n_outputs=5, #y_train.shape[1],
    module__n_extra_layers=3,
    optimizer=optim.SGD,
    criterion=nn.CrossEntropyLoss,
#    criterion=nn.MSELoss,
    lr=0.5,
    **niceties,
)

In [144]:
model.fit(X_train.compute(), y_train.compute())

  epoch    train_loss     dur
-------  ------------  ------
      1        1.6017  0.0246
      2        1.5993  0.0095
      3        1.5973  0.0073
      4        1.5955  0.0039
      5        1.5939  0.0046
      6        1.5924  0.0599
      7        1.5908  0.0414
      8        1.5892  0.0104
      9        1.5876  0.0042
     10        1.5860  0.0246
     11        1.5842  0.0491
     12        1.5824  0.0118
     13        1.5804  0.0238
     14        1.5782  0.0167
     15        1.5759  0.0773
     16        1.5733  0.0055
     17        1.5705  0.0058
     18        1.5673  0.0138
     19        1.5639  0.0233
     20        1.5600  0.0595
     21        1.5556  0.0095
     22        1.5506  0.0046
     23        1.5448  0.0035
     24        1.5383  0.0158
     25        1.5309  0.0050
     26        1.5224  0.0374
     27        1.5129  0.0349
     28        1.5022  0.0083
     29        1.4902  0.0070
     30        1.4768  0.0151
     31        1.4620  0.0111
     32   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=10, out_features=100, bias=True)
    (extra_layers): ModuleList(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Linear(in_features=100, out_features=100, bias=True)
      (2): Linear(in_features=100, out_features=100, bias=True)
    )
    (fc2): Linear(in_features=100, out_features=5, bias=True)
  ),
)

In [145]:
model.module_.extra_layers

ModuleList(
  (0): Linear(in_features=100, out_features=100, bias=True)
  (1): Linear(in_features=100, out_features=100, bias=True)
  (2): Linear(in_features=100, out_features=100, bias=True)
)

In [146]:
model.module_.parameters

<bound method Module.parameters of HiddenLayerNet(
  (fc1): Linear(in_features=10, out_features=100, bias=True)
  (extra_layers): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=100, bias=True)
  )
  (fc2): Linear(in_features=100, out_features=5, bias=True)
)>

In [150]:
list(zip(model.predict(X_train.compute()), y_train.compute()))

[(0, 0),
 (4, 4),
 (4, 4),
 (1, 1),
 (3, 3),
 (1, 1),
 (4, 4),
 (3, 3),
 (3, 3),
 (2, 2),
 (4, 4),
 (3, 3),
 (1, 1),
 (2, 2),
 (1, 1),
 (2, 2),
 (2, 2),
 (0, 0),
 (2, 2),
 (3, 3),
 (2, 2),
 (2, 2),
 (1, 1),
 (2, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (2, 2),
 (0, 0),
 (1, 1),
 (3, 3),
 (1, 1),
 (2, 2),
 (1, 1),
 (3, 3),
 (0, 0),
 (4, 4),
 (0, 0),
 (4, 4),
 (4, 4),
 (3, 3),
 (2, 2),
 (4, 4),
 (3, 3),
 (1, 1),
 (4, 4),
 (4, 4),
 (1, 1),
 (2, 1),
 (1, 1),
 (2, 2),
 (4, 4),
 (3, 3),
 (4, 4),
 (2, 2),
 (3, 3),
 (4, 4),
 (1, 1),
 (4, 4),
 (2, 2),
 (2, 2),
 (0, 0),
 (2, 2),
 (4, 4),
 (3, 3),
 (0, 0),
 (0, 0),
 (4, 4),
 (2, 1),
 (2, 2),
 (4, 4),
 (0, 0),
 (1, 1),
 (3, 3),
 (0, 0),
 (3, 3),
 (1, 1),
 (0, 0),
 (4, 4),
 (4, 4),
 (1, 1),
 (0, 0),
 (3, 3),
 (1, 1),
 (2, 2),
 (4, 4),
 (0, 0),
 (2, 2),
 (4, 4)]

In [148]:
model.score(X_train.compute(), y_train.compute())

0.9666666666666667

## Test Hyperparam Search

In [16]:
from skorch import NeuralNetClassifier

niceties = {
    "callbacks": False,
    "warm_start": True,
    "train_split": None,
    "max_epochs": 1,
}

model = NeuralNetClassifier(
    module=HiddenLayerNet,
    module__n_features=X_train.shape[1],
    module__n_outputs=5, #y_train.shape[1],
    module__n_extra_layers=3,
    optimizer=optim.SGD,
    criterion=nn.CrossEntropyLoss,
#    criterion=nn.MSELoss,
    lr=0.5,
    **niceties,
)

In [17]:
from scipy.stats import loguniform, uniform

params = {
    "module__activation": ["relu", "elu", "softsign", "leaky_relu", "rrelu"],
    "batch_size": [32, 64, 128, 256],
    "optimizer__lr": loguniform(1e-4, 1e-3),
    "optimizer__weight_decay": loguniform(1e-6, 1e-3),
    "optimizer__momentum": uniform(0, 1),
    "optimizer__nesterov": [True],
}

In [18]:
from dask_ml.model_selection import HyperbandSearchCV
search = HyperbandSearchCV(model, params, random_state=42, verbose=True, max_iter=9)

In [19]:
#y_train2 = y_train.reshape(-1, 1).persist()
y_train2 = y_train.persist()
_ = search.fit(X_train, y_train2)

[CV, bracket=2] creating 9 models
[CV, bracket=1] creating 5 models
[CV, bracket=0] creating 3 models
[CV, bracket=0] For training there are between 7200 and 7200 examples in each chunk
[CV, bracket=1] For training there are between 7200 and 7200 examples in each chunk
[CV, bracket=2] For training there are between 7200 and 7200 examples in each chunk
[CV, bracket=0] validation score of 0.1978 received after 1 partial_fit calls
[CV, bracket=1] validation score of 0.2006 received after 1 partial_fit calls
[CV, bracket=2] validation score of 0.2061 received after 1 partial_fit calls
[CV, bracket=1] validation score of 0.2011 received after 3 partial_fit calls
[CV, bracket=2] validation score of 0.2044 received after 3 partial_fit calls
[CV, bracket=1] validation score of 0.2044 received after 9 partial_fit calls
[CV, bracket=2] validation score of 0.2006 received after 9 partial_fit calls


In [20]:
search.best_score_

0.20444444444444446

In [21]:
search.best_params_

{'batch_size': 128,
 'module__activation': 'rrelu',
 'optimizer__lr': 0.0003961342081293917,
 'optimizer__momentum': 0.3989504451091438,
 'optimizer__nesterov': True,
 'optimizer__weight_decay': 4.254186565699362e-05}

In [22]:
search.best_estimator_

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=10, out_features=100, bias=True)
    (extra_layers): ModuleList(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Linear(in_features=100, out_features=100, bias=True)
      (2): Linear(in_features=100, out_features=100, bias=True)
    )
    (fc2): Linear(in_features=100, out_features=5, bias=True)
  ),
)

In [23]:
from dask_ml.wrappers import ParallelPostFit
deployed_model = ParallelPostFit(search.best_estimator_)

In [24]:
deployed_model.score(X_test, y_test)

0.217

In [27]:
deployed_model.score(X_train, y_train)

0.1998888888888889

In [25]:
deployed_model.predict(X_train).compute()

array([0, 4, 0, ..., 2, 2, 4])

In [26]:
y_train.compute()

array([4, 1, 3, ..., 0, 0, 1])

# Amazon Train Set (small)

Dataset here is from:
https://course.fast.ai/datasets

In [19]:
!ls /workspace/data

amazon_review_full_csv.tgz  train_small.tsv


In [14]:
import dask.dataframe as dd

url="/workspace/data/train_small.tsv"

columns=["review_score", "title", "text"]
categorical = ["review_score"]

df = dd.read_csv(
    url,
    names=columns,
    dtype={
        "review_score": "UInt8",
        "title": "string",
        "text": "string"
    },
    #blocksize="16 MiB",
    blocksize="128 KiB"
)

In [15]:
df

,review_score,title,text
npartitions=35,,,
,UInt8,string,string
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [16]:
df.head()

,review_score,title,text
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


In [17]:
# This is an input, not an output so don't need to do this!


# one hot encode the categorical columns;
# if df["foo"].unique() == [1, 3, 4], add columns foo_1, foo_3, foo_4
#df = dd.get_dummies(df, columns=categorical)

# persist so only download once
df = df.persist()

In [18]:
df.head()

,review_score,title,text
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


## Local Feature Computation

In [40]:
sent_list = df["text"].compute().tolist()

In [41]:
sent_series = df["text"].compute()

In [16]:
sent_series[:100]

0     Gave this to my dad for a gag gift after direc...
1     I hope a lot of people hear this cd. We need m...
2     I'm reading a lot of reviews saying that this ...
3     The music of Yasunori Misuda is without questi...
4     Probably the greatest soundtrack in history! U...
                            ...                        
95    I agree, the CDs are a much needed help, since...
96    I recommend this book highly. I have a competi...
97    This is an AWESOME game! Almost everyone know ...
98    I bought this game to bring down to my vacatio...
99    After doing alot of researching and reading re...
Name: text, Length: 100, dtype: string

In [35]:
num_samples=10000

In [12]:
embeddings_file="./embeddings_%d_samples.npy"%num_samples

### Sentence Embeddings

In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [92]:
str(model)

'SentenceTransformer(\n  (0): BERT(\n    (bert): BertModel(\n      (embeddings): BertEmbeddings(\n        (word_embeddings): Embedding(30522, 768, padding_idx=0)\n        (position_embeddings): Embedding(512, 768)\n        (token_type_embeddings): Embedding(2, 768)\n        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)\n        (dropout): Dropout(p=0.1, inplace=False)\n      )\n      (encoder): BertEncoder(\n        (layer): ModuleList(\n          (0): BertLayer(\n            (attention): BertAttention(\n              (self): BertSelfAttention(\n                (query): Linear(in_features=768, out_features=768, bias=True)\n                (key): Linear(in_features=768, out_features=768, bias=True)\n                (value): Linear(in_features=768, out_features=768, bias=True)\n                (dropout): Dropout(p=0.1, inplace=False)\n              )\n              (output): BertSelfOutput(\n                (dense): Linear(in_features=768, out_features=768, bias=True

In [138]:
%%time
sentence_embeddings = model.encode(sent_list[:300], batch_size=50)

CPU times: user 1min 35s, sys: 6.74 s, total: 1min 41s
Wall time: 25.9 s


In [15]:
import torch

In [16]:
torch.get_num_threads()

4

In [18]:
torch.set_num_threads(1)

In [19]:
torch.get_num_threads()

1

In [141]:
%%time
sentence_embeddings = model.encode(sent_list[:300], batch_size=50)

CPU times: user 1min 14s, sys: 4.63 s, total: 1min 19s
Wall time: 39.8 s


In [25]:
sentence_embeddings.shape

(1000, 768)

In [2]:
%%time
print(num_samples)
sentence_embeddings = model.encode(sent_list[:num_samples], batch_size=250)

NameError: name 'num_samples' is not defined

In [74]:
# save tehse so we don't have to keep doing this
np.save(embeddings_file, sentence_embeddings)

In [76]:
sentence_embeddings = np.load(embeddings_file)

In [77]:
sentence_embeddings.shape

(1000, 768)

### Tfidf

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english', max_features=20000)

In [44]:
tfidfX = np.array(vectorizer.fit_transform(sent_list[:num_samples]).todense().astype('float32'))

In [45]:
tfidfX.shape

(10000, 20000)

In [46]:
type(tfidfX)

numpy.ndarray

### Outputs

In [47]:
classes = np.array(df["review_score"].compute().tolist()).astype('long')
classes = classes[:num_samples]-1

In [48]:
classes

array([2, 4, 4, ..., 0, 0, 3])

## Dask computation

In [19]:
import importlib

In [20]:
client.upload_file("transformer_wrapper.py")

In [21]:
import transformer_wrapper
importlib.reload(transformer_wrapper)
from transformer_wrapper import check_models, load_model, gen_sent_features

In [22]:
client.run(check_models)

{'tcp://172.18.0.3:44975': ['public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip']}

In [23]:
client.run(load_model)

{'tcp://172.18.0.3:44975': (2.1749770641326904,
  'SentenceTransformer(\n  (0): BERT(\n    (bert): BertModel(\n      (embeddings): BertEmbeddings(\n        (word_embeddings): Embedding(30522, 768, padding_idx=0)\n        (position_embeddings): Embedding(512, 768)\n        (token_type_embeddings): Embedding(2, 768)\n        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)\n        (dropout): Dropout(p=0.1, inplace=False)\n      )\n      (encoder): BertEncoder(\n        (layer): ModuleList(\n          (0): BertLayer(\n            (attention): BertAttention(\n              (self): BertSelfAttention(\n                (query): Linear(in_features=768, out_features=768, bias=True)\n                (key): Linear(in_features=768, out_features=768, bias=True)\n                (value): Linear(in_features=768, out_features=768, bias=True)\n                (dropout): Dropout(p=0.1, inplace=False)\n              )\n              (output): BertSelfOutput(\n                (dense): L

In [24]:
d_gen_sent_features = dask.delayed(gen_sent_features)

In [25]:
%%time
d_gen_sent_features(sent_list[:10]).compute()

NameError: name 'sent_list' is not defined

In [152]:
%%time
d_gen_sent_features(sent_list[:300]).compute()

CPU times: user 41.3 ms, sys: 10.1 ms, total: 51.5 ms
Wall time: 48.7 s


array([[-0.4301419 ,  0.635654  ,  1.0057247 , ...,  0.3879471 ,
        -0.32287857,  0.29900575],
       [-0.15508707, -0.40734345,  0.33508727, ...,  0.42280957,
        -0.19019693, -0.51367515],
       [ 0.1111841 ,  0.5360189 ,  0.17411903, ...,  0.6714969 ,
         0.15350488,  0.1286595 ],
       ...,
       [-0.46399707,  0.4504652 ,  0.8953199 , ...,  0.62237537,
         0.04999361,  0.03117059],
       [-0.8013059 ,  0.33056563,  0.08894264, ..., -0.2287397 ,
         0.8540461 ,  0.17452975],
       [-0.4195155 , -0.08595073,  0.2911015 , ...,  1.0259533 ,
        -0.1021437 ,  0.05288688]], dtype=float32)

In [82]:
%%time
d_gen_sent_features(sent_list[:1000]).compute()

CPU times: user 154 ms, sys: 23.9 ms, total: 178 ms
Wall time: 2min 24s


array([[-0.4301419 ,  0.635654  ,  1.0057247 , ...,  0.3879471 ,
        -0.32287857,  0.29900575],
       [-0.15508707, -0.40734345,  0.33508727, ...,  0.42280957,
        -0.19019693, -0.51367515],
       [ 0.11118428,  0.536019  ,  0.17411919, ...,  0.67149687,
         0.15350482,  0.12865965],
       ...,
       [-0.38612026,  0.84072685,  0.12853882, ...,  0.6550069 ,
        -0.06093563, -0.07394619],
       [-0.45464376,  0.4275732 ,  0.02125585, ...,  0.7673474 ,
        -0.40999392,  0.365916  ],
       [-0.30104864,  1.0877458 , -0.27813938, ...,  0.42075455,
         0.08935574,  0.33397803]], dtype=float32)

In [113]:
feat = df[df['review_score'] == 1].text.map_partitions(gen_sent_features, 
                                                       meta=np.empty((0,0), dtype='float32'))

In [115]:
feat.compute_chunk_sizes()

dask.array<gen_sent_features, shape=(2003, 768), dtype=float32, chunksize=(2003, 768), chunktype=numpy.ndarray>

In [116]:
feat[:10].compute()

KeyboardInterrupt: 

In [36]:
len(df.text)

10000

In [26]:
feat = df.text.map_partitions(gen_sent_features, meta=np.empty((0,0), 
                                dtype='float32'))

In [27]:
sentence_embeddings = feat.compute()

In [28]:
embeddings_file_full = "embeddings_full.npy"

In [30]:
# save tehse so we don't have to keep doing this
np.save(embeddings_file_full, sentence_embeddings)

In [31]:
sentence_embeddings.shape

(10000, 768)

## Local Model Training

### With Embeddings

In [73]:
from dask_ml.model_selection import train_test_split
import dask

X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings, classes, shuffle=True)

# persist the data so it's not re-computed
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [74]:
X_train

array([[-0.6420861 ,  0.35635602,  0.46377364, ..., -0.02072408,
        -0.15344056,  0.5135324 ],
       [-0.27175313, -0.09459253,  1.2993565 , ...,  0.81999063,
         0.28646803,  0.62803245],
       [-0.66346896,  0.18068172,  0.6943203 , ...,  0.9732542 ,
        -0.27000448,  0.26392078],
       ...,
       [-0.7146554 ,  1.0973696 ,  0.8692502 , ...,  0.78516215,
        -0.08688027,  0.28347337],
       [ 0.11670861,  0.41923362,  0.5678272 , ...,  0.25726083,
         0.01891077,  0.29377976],
       [-0.34464687,  0.2876471 ,  0.45760393, ...,  0.45491216,
        -0.73659074, -0.0227952 ]], dtype=float32)

In [75]:
y_train

array([2, 1, 4, ..., 0, 2, 1])

In [76]:
import importlib
import torch_model

importlib.reload(torch_model)

# Import our HiddenLayerNet pytorch model from a local torch_model.py module
from torch_model import HiddenLayerNet
# Send module with HiddenLayerNet to workers on cluster
client.upload_file("torch_model.py")

In [77]:
from skorch import NeuralNetClassifier

niceties = {
    "callbacks": False,
    "warm_start": True,
    "train_split": None,
    "max_epochs": 100,
}

model = NeuralNetClassifier(
    module=HiddenLayerNet,
    module__n_features=X_train.shape[1],
    module__n_outputs=5, #y_train.shape[1],
    module__n_extra_layers=3,
    optimizer=optim.SGD,
    criterion=nn.CrossEntropyLoss,
#    criterion=nn.MSELoss,
    lr=0.1,
    **niceties,
)

In [78]:
model.fit(X_train, y_train)

  epoch    train_loss     dur
-------  ------------  ------
      1        1.6043  0.3022
      2        1.4560  0.2525
      3        1.3229  0.2537
      4        1.2841  0.2861
      5        1.2499  0.3312
      6        1.2307  0.2674
      7        1.2186  0.3187
      8        1.2086  0.2676
      9        1.1992  0.2661
     10        1.1902  0.2646
     11        1.1825  0.2760
     12        1.1744  0.2666
     13        1.1686  0.2382
     14        1.1629  0.2324
     15        1.1569  0.2734
     16        1.1487  0.2675
     17        1.1432  0.2570
     18        1.1339  0.2548
     19        1.1306  0.2692
     20        1.1194  0.2484
     21        1.1200  0.2702
     22        1.1136  0.3004
     23        1.1008  0.2679
     24        1.1027  0.2621
     25        1.0924  0.2521
     26        1.0833  0.3199
     27        1.0860  0.2822
     28        1.0741  0.2332
     29        1.0583  0.3073
     30        1.0609  0.2812
     31        1.0604  0.2662
     32   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=768, out_features=100, bias=True)
    (extra_layers): ModuleList(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Linear(in_features=100, out_features=100, bias=True)
      (2): Linear(in_features=100, out_features=100, bias=True)
    )
    (fc2): Linear(in_features=100, out_features=5, bias=True)
  ),
)

In [79]:
model.module_.parameters

<bound method Module.parameters of HiddenLayerNet(
  (fc1): Linear(in_features=768, out_features=100, bias=True)
  (extra_layers): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=100, bias=True)
  )
  (fc2): Linear(in_features=100, out_features=5, bias=True)
)>

In [80]:
list(zip(model.predict(X_train), y_train))

[(2, 2),
 (1, 1),
 (4, 4),
 (3, 3),
 (2, 2),
 (4, 4),
 (1, 1),
 (0, 1),
 (1, 0),
 (1, 1),
 (3, 3),
 (4, 4),
 (4, 4),
 (2, 2),
 (4, 4),
 (2, 2),
 (3, 3),
 (4, 4),
 (0, 0),
 (1, 2),
 (0, 0),
 (4, 4),
 (4, 4),
 (1, 1),
 (2, 2),
 (1, 1),
 (0, 0),
 (3, 3),
 (4, 4),
 (3, 2),
 (1, 1),
 (2, 1),
 (4, 4),
 (2, 3),
 (3, 3),
 (1, 1),
 (2, 2),
 (1, 1),
 (3, 4),
 (1, 1),
 (3, 3),
 (2, 2),
 (0, 2),
 (1, 1),
 (4, 4),
 (2, 1),
 (4, 4),
 (1, 1),
 (4, 4),
 (0, 0),
 (2, 2),
 (1, 1),
 (1, 1),
 (3, 3),
 (1, 1),
 (1, 1),
 (3, 3),
 (0, 0),
 (0, 0),
 (4, 4),
 (3, 3),
 (3, 3),
 (2, 2),
 (4, 4),
 (1, 1),
 (0, 1),
 (4, 4),
 (1, 0),
 (1, 1),
 (3, 3),
 (0, 0),
 (1, 1),
 (4, 4),
 (1, 0),
 (3, 3),
 (4, 4),
 (0, 0),
 (3, 4),
 (3, 3),
 (4, 3),
 (2, 2),
 (2, 2),
 (1, 1),
 (1, 1),
 (2, 3),
 (1, 1),
 (4, 4),
 (1, 1),
 (4, 4),
 (3, 3),
 (2, 2),
 (1, 1),
 (3, 3),
 (2, 2),
 (1, 1),
 (1, 1),
 (0, 0),
 (3, 3),
 (2, 2),
 (2, 2),
 (4, 4),
 (3, 3),
 (0, 0),
 (3, 3),
 (4, 4),
 (1, 1),
 (4, 4),
 (4, 4),
 (1, 1),
 (1, 1),
 (0, 0),
 

In [81]:
model.score(X_train, y_train)

0.7513333333333333

In [82]:
model.score(X_test, y_test)

0.379

In [83]:
X_train.shape

(9000, 768)

In [84]:
confusion_matrix(y_test, model.predict(X_test))

array([[69, 69, 29, 21,  9],
       [34, 89, 70, 22,  5],
       [ 6, 76, 76, 50,  5],
       [ 8,  4, 54, 90, 32],
       [ 4, 11, 23, 89, 55]])

### Tfidf Features

In [60]:
from dask_ml.model_selection import train_test_split
import dask

X_train, X_test, y_train, y_test = train_test_split(tfidfX, classes, shuffle=True)

# persist the data so it's not re-computed
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [61]:
X_train.shape

(9000, 20000)

In [58]:
np.argmax(X_train, axis=1)

array([2742, 3259, 7853, 1161, 3807, 3574, 3780, 3389,  681, 5280,  312,
       4469, 7625, 3333, 6016, 1414,  809, 4720,  117, 1774,  926, 2967,
       3262, 1606, 3741, 7509, 7686, 3274, 7586,  349, 6399,  676, 1284,
       6976, 2413, 2676, 6735, 5350,  178, 5475, 6788, 5101, 6314, 1399,
        534, 3346, 4810, 2083, 5816, 1519, 6484, 5658, 1396, 4475, 4126,
       1823, 2952,  330, 7254, 1819,  744, 3012, 6092, 7841, 1430, 4111,
       1616, 2667, 5036, 6435, 3916, 2527, 7864, 5411, 1912, 1410, 3028,
       2200,  666, 6735, 3373, 1138, 3996,  227,  926, 2756,  430, 3586,
       6646, 3296, 5317, 6734, 5404, 7761,  463, 5703,  989, 7654,  442,
       1828, 1209, 6454, 4809, 5891,  814,  347, 5254, 4926, 3510, 2552,
       4960, 2281, 7205,  499, 3769,  878, 1325, 7698,  167, 2431, 7555,
        249, 7837, 7112, 1051,   36, 4353, 5101, 4605, 2501, 6090, 3406,
       6453, 1153, 1252, 1324, 1808,  976, 4251, 6455, 3195,  749, 1229,
        372,  829, 3333, 5608, 3888, 4247,  107,  7

In [46]:
y_train

array([1, 3, 3, 2, 1, 2, 1, 2, 2, 2, 0, 0, 4, 0, 3, 4, 1, 0, 1, 1, 3, 2,
       4, 2, 0, 1, 1, 1, 2, 4, 2, 3, 1, 3, 3, 3, 2, 2, 0, 1, 0, 3, 4, 0,
       1, 3, 4, 1, 3, 2, 2, 2, 2, 2, 0, 2, 0, 0, 3, 2, 4, 1, 0, 4, 3, 3,
       3, 2, 3, 4, 3, 3, 2, 1, 2, 1, 3, 3, 4, 1, 3, 3, 3, 0, 4, 0, 1, 3,
       0, 0, 4, 4, 2, 4, 3, 1, 4, 3, 3, 2, 2, 2, 3, 4, 0, 4, 3, 1, 0, 3,
       1, 0, 3, 1, 0, 4, 1, 0, 4, 2, 0, 3, 4, 3, 0, 1, 4, 2, 2, 4, 1, 0,
       4, 2, 2, 2, 3, 2, 0, 1, 4, 0, 3, 4, 4, 1, 1, 1, 3, 4, 2, 4, 1, 3,
       3, 1, 2, 0, 1, 0, 1, 4, 2, 4, 2, 0, 2, 2, 1, 3, 4, 0, 1, 4, 4, 4,
       2, 3, 2, 0, 4, 3, 1, 0, 1, 1, 3, 2, 3, 2, 4, 3, 2, 4, 4, 0, 3, 0,
       1, 2, 0, 2, 0, 2, 4, 4, 2, 3, 1, 0, 2, 0, 2, 2, 2, 3, 0, 3, 2, 1,
       2, 4, 2, 2, 3, 3, 4, 3, 1, 2, 3, 1, 3, 3, 1, 1, 0, 4, 2, 3, 3, 0,
       4, 3, 2, 2, 2, 3, 0, 0, 2, 1, 1, 3, 4, 1, 0, 4, 0, 3, 4, 2, 4, 0,
       0, 2, 0, 2, 0, 3, 1, 0, 0, 2, 3, 0, 2, 1, 4, 3, 4, 4, 3, 3, 2, 0,
       2, 4, 1, 4, 0, 1, 4, 3, 1, 2, 4, 3, 2, 2, 0,

In [62]:
import importlib
import torch_model

importlib.reload(torch_model)

# Import our HiddenLayerNet pytorch model from a local torch_model.py module
from torch_model import HiddenLayerNet
# Send module with HiddenLayerNet to workers on cluster
client.upload_file("torch_model.py")

In [63]:
from skorch import NeuralNetClassifier

niceties = {
    "callbacks": False,
    "warm_start": True,
    "train_split": None,
    "max_epochs": 100,
}

model = NeuralNetClassifier(
    module=HiddenLayerNet,
    module__n_features=X_train.shape[1],
    module__n_outputs=5, #y_train.shape[1],
    module__n_extra_layers=0,
    optimizer=optim.SGD,
    criterion=nn.CrossEntropyLoss,
#    criterion=nn.MSELoss,
    lr=0.5,
    **niceties,
)

In [64]:
model.fit(X_train, y_train)

  epoch    train_loss     dur
-------  ------------  ------
      1        1.6101  1.1552
      2        1.6081  1.2562
      3        1.6045  1.1296
      4        1.5976  1.1284
      5        1.5841  1.1832
      6        1.5587  1.1678
      7        1.5169  1.1704
      8        1.4626  1.1323
      9        1.4231  1.3477
     10        1.4015  1.1582
     11        1.3631  1.1699
     12        1.3230  1.3124
     13        1.2928  1.1512
     14        1.2690  1.0116
     15        1.2344  1.1215
     16        1.1971  1.1347
     17        1.1507  1.1438
     18        1.1517  1.1394
     19        1.1076  1.1139
     20        1.1036  1.1442
     21        1.0318  1.1421
     22        1.0098  1.1672
     23        0.9705  1.0916
     24        1.0065  1.1335
     25        0.9673  1.1170
     26        0.9646  1.1165
     27        0.9597  1.1092
     28        0.9086  1.0962
     29        0.8674  1.0711
     30        0.8607  1.0104
     31        0.8312  1.0795
     32   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=20000, out_features=100, bias=True)
    (extra_layers): ModuleList()
    (fc2): Linear(in_features=100, out_features=5, bias=True)
  ),
)

In [61]:
model.module_.extra_layers

ModuleList()

In [65]:
model.module_.parameters

<bound method Module.parameters of HiddenLayerNet(
  (fc1): Linear(in_features=20000, out_features=100, bias=True)
  (extra_layers): ModuleList()
  (fc2): Linear(in_features=100, out_features=5, bias=True)
)>

In [66]:
list(zip(model.predict(X_train), y_train))

[(1, 1),
 (2, 2),
 (0, 0),
 (0, 0),
 (4, 4),
 (4, 4),
 (2, 2),
 (1, 1),
 (3, 3),
 (0, 0),
 (2, 2),
 (4, 4),
 (4, 4),
 (4, 4),
 (4, 4),
 (2, 2),
 (3, 3),
 (4, 4),
 (2, 2),
 (4, 4),
 (2, 2),
 (3, 3),
 (0, 0),
 (0, 0),
 (2, 2),
 (0, 0),
 (0, 0),
 (1, 1),
 (3, 3),
 (2, 2),
 (1, 1),
 (4, 4),
 (0, 0),
 (0, 0),
 (4, 4),
 (2, 2),
 (1, 1),
 (0, 0),
 (3, 3),
 (1, 1),
 (4, 4),
 (3, 3),
 (3, 3),
 (1, 1),
 (1, 1),
 (3, 3),
 (2, 2),
 (4, 4),
 (2, 2),
 (3, 3),
 (2, 2),
 (1, 1),
 (3, 3),
 (1, 1),
 (2, 2),
 (2, 2),
 (4, 4),
 (2, 2),
 (2, 2),
 (2, 2),
 (3, 3),
 (1, 1),
 (3, 3),
 (1, 1),
 (0, 0),
 (0, 0),
 (4, 4),
 (3, 3),
 (3, 3),
 (0, 0),
 (2, 2),
 (4, 4),
 (4, 4),
 (1, 1),
 (4, 4),
 (3, 3),
 (0, 0),
 (3, 3),
 (1, 1),
 (3, 3),
 (3, 3),
 (3, 3),
 (2, 2),
 (0, 0),
 (2, 2),
 (2, 2),
 (3, 3),
 (0, 0),
 (4, 4),
 (0, 0),
 (2, 2),
 (0, 0),
 (0, 0),
 (4, 4),
 (1, 1),
 (1, 1),
 (4, 4),
 (4, 4),
 (1, 1),
 (2, 2),
 (1, 1),
 (0, 0),
 (3, 3),
 (1, 1),
 (0, 0),
 (3, 3),
 (0, 0),
 (1, 1),
 (1, 1),
 (4, 4),
 (0, 0),
 

In [67]:
model.score(X_train, y_train)

0.9998888888888889

In [68]:
model.score(X_test, y_test)

0.39

In [72]:
confusion_matrix(y_test, model.predict(X_test))

array([[ 86,  56,  22,   4,  23],
       [ 49,  77,  40,  12,  20],
       [ 26,  45,  59,  52,  30],
       [ 13,  19,  48,  58,  71],
       [ 11,  13,  17,  39, 110]])

# Coiled Computation

In [75]:
import coiled

In [76]:
coiled.__version__

'0.0.21'

## Setup Cluster

In [3]:
# this is based off coied-default with some added stuff like pytorch and skorch
conda_env = {
  "channels": [
    "conda-forge",
    "defaults"
  ],
  "dependencies": [
    "bokeh>=2.1.1",
    "bottleneck",
    "cytoolz",
    "dask-image>=0.3.0",
    "dask-ml>=1.5.0",
    "dask=2.21.0",
    "h5py",
    "lz4",
    "numba",
    "numpy>=1.19.0",
    "pandas>=1.0.5",
    "pillow>=7.2.0",
    "pip",
    "pyarrow>=0.15.1",
    "python-blosc",
    "python-graphviz",
    "python=3.8",
    "pytorch",
    "skorch",
    "requests",
    "s3fs",
    "scikit-learn>=0.23.1",
    "xarray",    
  ]
}

pip_env = [
    # useful to generate features for models
    "sentence-transformers"
]

In [4]:
#coiled.delete_cluster_configuration("ml_gpu")
coiled.create_cluster_configuration(
    name="ml_gpu",
#    software="def_pytorch",
#    software="pip_only_gpu",
    software="gpu_pytorch",
    worker_memory="16 GiB",
    worker_gpu=1,
    scheduler_memory="4 GiB",
    scheduler_cpu=1,
)

In [5]:
cluster_name="nlp_gpu_cluster"

In [18]:
coiled.delete_cluster(cluster_name)

In [6]:
cluster = coiled.Cluster(
    name = cluster_name,
    n_workers=1, 
    configuration="ml_gpu"
#    configuration="ml_cpu"
)
client = dask.distributed.Client(cluster)

client

Creating Cluster. This takes about a minute ...

KeyboardInterrupt: 

In [77]:
cluster_name="nlp_cpu_cluster1"

In [9]:
coiled.delete_cluster(cluster_name)

In [25]:
t0=time.time()
cluster = coiled.Cluster(
    name = cluster_name,
    n_workers=1, 
#    configuration="ml_gpu"
    configuration="ml_cpu"
)
t1=time.time()
print("%.3fs to start cluster\n"%(t1-t0))

192.793s to start clusterkes about a minute ...


In [78]:
t0=time.time()
cluster = coiled.Cluster(
    name = cluster_name,
    n_workers=1, 
#    configuration="ml_gpu"
    configuration="ml_cpu"
)
t1=time.time()
print("%.3fs to start cluster\n"%(t1-t0))

188.235s to start clusterkes about a minute ...



In [79]:
client = dask.distributed.Client(cluster)

client

/workspace/src/dask/distributed/distributed/client.py:1130: VersionMismatchWarning: Mismatched versions found

+-------------+--------------------+-----------+---------+
| Package     | client             | scheduler | workers |
+-------------+--------------------+-----------+---------+
| dask        | 2.19.0+2.g4d7e6741 | 2.21.0    | 2.21.0  |
| distributed | 2.19.0             | 2.22.0    | 2.22.0  |
+-------------+--------------------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tls://ec2-3-129-10-41.us-east-2.compute.amazonaws.com:8786 Dashboard: http://ec2-3-129-10-41.us-east-2.compute.amazonaws.com:8787/status,Cluster Workers: 1 Cores: 4 Memory: 17.18 GB


## Test GPU Status

In [29]:
import importlib
import torch_model

importlib.reload(torch_model)

# Import our HiddenLayerNet pytorch model from a local torch_model.py module
from torch_model import HiddenLayerNet
# Send module with HiddenLayerNet to workers on cluster
client.upload_file("torch_model.py")

In [30]:
import transformer_wrapper
importlib.reload(transformer_wrapper)
from transformer_wrapper import check_models, load_model, gen_sent_features, check_gpu_access

In [38]:
client.upload_file("transformer_wrapper.py")

In [39]:
client.run(check_gpu_access)

{'tls://10.1.11.119:33555': False,
 'tls://10.1.11.135:41187': False,
 'tls://10.1.11.167:35175': False,
 'tls://10.1.11.229:33105': False,
 'tls://10.1.11.57:44587': False,
 'tls://10.1.11.85:35517': False,
 'tls://10.1.11.88:42941': False,
 'tls://10.1.12.101:42771': False,
 'tls://10.1.12.179:34217': False,
 'tls://10.1.12.221:41673': False,
 'tls://10.1.12.246:34075': False,
 'tls://10.1.12.54:38741': False,
 'tls://10.1.12.62:43131': False,
 'tls://10.1.12.96:35289': False,
 'tls://10.1.13.102:34127': False,
 'tls://10.1.13.113:39655': False,
 'tls://10.1.13.15:41881': False,
 'tls://10.1.13.187:35913': False,
 'tls://10.1.13.31:44427': False,
 'tls://10.1.13.58:44211': False}

## Upload Data

In [80]:
cluster.scale(20)

In [81]:
num_samples=10000

In [ ]:
# we need to set up a public bucket with this dataset to make this work
import dask.dataframe as dd

url="/workspace/data/train_small.tsv"

columns=["review_score", "title", "text"]
categorical = ["review_score"]

df = dd.read_csv(
    url,
    names=columns,
    dtype={
        "review_score": "UInt8",
        "title": "string",
        "text": "string"
    },
    #blocksize="16 MiB",
    blocksize="128 KiB"
)

In [5]:
import pandas as pd

url="/workspace/data/train_small.tsv"

columns=["review_score", "title", "text"]
categorical = ["review_score"]

p_df = pd.read_csv(
    url,
    names=columns,
    dtype={
        "review_score": "UInt8",
        "title": "string",
        "text": "string"
    },
)

In [6]:
nchunks=8*10

In [36]:
ddf = dd.from_pandas(p_df[:num_samples], npartitions=nchunks)
ddf.persist()

,review_score,title,text
npartitions=80,,,
0,UInt8,string,string
125,...,...,...
...,...,...,...
9875,...,...,...
9999,...,...,...


### Outputs

In [10]:
classes = np.array(p_df["review_score"].tolist()).astype('long')
classes = classes[:num_samples]-1

In [11]:
classes

array([2, 4, 4, ..., 0, 0, 3])

In [71]:
# we subtract 1 so that classes are 0-indexed
d_classes = (ddf['review_score'].astype("long") - 1).persist()

In [66]:
d_classes.compute()

0       2
1       4
2       4
3       3
4       4
       ..
9995    3
9996    2
9997    0
9998    0
9999    3
Name: review_score, Length: 10000, dtype: int64

In [82]:
# actually we need an array and it seems a bit painful to convert a dask series 
# to a dask aray directly so doing it locally
#d_classes = da.from_array(classes, chunks=nchunks).reshape((num_samples,1)).persist()
d_classes = da.from_array(classes, chunks=nchunks).persist()

In [83]:
d_classes

,Array,Chunk
Bytes,80.00 kB,640 B
Shape,"(10000,)","(80,)"
Count,125 Tasks,125 Chunks
Type,int64,numpy.ndarray


## Generate Embeddings

In [40]:
client.run(check_models)

{'tls://10.1.11.119:33555': [],
 'tls://10.1.11.135:41187': [],
 'tls://10.1.11.167:35175': [],
 'tls://10.1.11.229:33105': [],
 'tls://10.1.11.57:44587': [],
 'tls://10.1.11.85:35517': [],
 'tls://10.1.11.88:42941': [],
 'tls://10.1.12.101:42771': [],
 'tls://10.1.12.179:34217': [],
 'tls://10.1.12.221:41673': [],
 'tls://10.1.12.246:34075': [],
 'tls://10.1.12.54:38741': [],
 'tls://10.1.12.62:43131': [],
 'tls://10.1.12.96:35289': [],
 'tls://10.1.13.102:34127': [],
 'tls://10.1.13.113:39655': [],
 'tls://10.1.13.15:41881': [],
 'tls://10.1.13.187:35913': [],
 'tls://10.1.13.31:44427': [],
 'tls://10.1.13.58:44211': []}

In [41]:
# load the model on all workers
ret = client.run(load_model)

In [42]:
# check that the model has been downloaded and locally cached
client.run(check_models, modeldir="/root/.cache/torch/sentence_transformers/")

{'tls://10.1.11.119:33555': ['public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip'],
 'tls://10.1.11.135:41187': ['public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip'],
 'tls://10.1.11.167:35175': ['public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip'],
 'tls://10.1.11.229:33105': ['public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip'],
 'tls://10.1.11.57:44587': ['public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip'],
 'tls://10.1.11.85:35517': ['public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip'],
 'tls://10.1.11.88:42941': ['public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip'],
 'tls://10.1.12.101:42771': ['public.ukp.informatik

In [43]:
# a feature generation function we map over the data frame
# it takes a list of strings as inputs and ouputs a matrix of features, one row per string
d_gen_sent_features = dask.delayed(gen_sent_features)

In [55]:
feat = ddf.text.map_partitions(gen_sent_features,
                               meta=np.empty((0,0), dtype='float32'))

In [22]:
%%time

# time with one worker

# this is 10k sents over 4 cores
sent_embeddings = feat.compute()
sent_embeddings.shape

CPU times: user 6.66 s, sys: 2.09 s, total: 8.75 s
Wall time: 45min 22s


(10000, 768)

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f1c1275e130>
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [56]:
%%time

# time with 20 workers

# this is 10k sents over 80 cores
feat = feat.persist()

CPU times: user 25.3 ms, sys: 419 µs, total: 25.7 ms
Wall time: 25.4 ms


In [45]:
%%time

# time with 20 workers

sentence_embeddings = feat.compute()
sentence_embeddings.shape

CPU times: user 3.7 s, sys: 1.07 s, total: 4.77 s
Wall time: 3min 23s


(10000, 768)

In [14]:
embeddings_file_full = "embeddings_full_coiled.npy"

In [50]:
# save tehse so we don't have to keep doing this
np.save(embeddings_file_full, sentence_embeddings)

In [16]:
sentence_embeddings = np.load(embeddings_file_full)
sentence_embeddings.shape

(10000, 768)

## Train with Embeddings

In [56]:
nchunks

80

In [84]:
# if we've loaded the cached versions
d_sentence_embeddings = da.from_array(sentence_embeddings, chunks=(nchunks,sentence_embeddings.shape[1]))

In [67]:
d_sentence_embeddings = feat

In [85]:
d_sentence_embeddings.compute_chunk_sizes()

,Array,Chunk
Bytes,30.72 MB,245.76 kB
Shape,"(10000, 768)","(80, 768)"
Count,126 Tasks,125 Chunks
Type,float32,numpy.ndarray


In [86]:
d_classes.compute_chunk_sizes()

,Array,Chunk
Bytes,80.00 kB,640 B
Shape,"(10000,)","(80,)"
Count,125 Tasks,125 Chunks
Type,int64,numpy.ndarray


In [87]:
from dask_ml.model_selection import train_test_split
import dask

X_train, X_test, y_train, y_test = train_test_split(d_sentence_embeddings, d_classes, shuffle=True)

# persist the data so it's not re-computed
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [99]:
X_train

,Array,Chunk
Bytes,27.65 MB,221.18 kB
Shape,"(9000, 768)","(72, 768)"
Count,125 Tasks,125 Chunks
Type,float32,numpy.ndarray


In [89]:
y_train

,Array,Chunk
Bytes,72.00 kB,576 B
Shape,"(9000,)","(72,)"
Count,125 Tasks,125 Chunks
Type,int64,numpy.ndarray


In [100]:
y_train.compute()

array([0, 1, 4, ..., 0, 1, 0])

In [90]:
import importlib
import torch_model

importlib.reload(torch_model)

# Import our HiddenLayerNet pytorch model from a local torch_model.py module
from torch_model import HiddenLayerNet
# Send module with HiddenLayerNet to workers on cluster
client.upload_file("torch_model.py")

In [91]:
from skorch import NeuralNetClassifier

niceties = {
    "callbacks": False,
    "warm_start": True,
    "train_split": None,
    "max_epochs": 100,
}

model = NeuralNetClassifier(
    module=HiddenLayerNet,
    module__n_features=X_train.shape[1],
    module__n_outputs=5, #y_train.shape[1],
    module__n_extra_layers=3,
    optimizer=optim.SGD,
    criterion=nn.CrossEntropyLoss,
#    criterion=nn.MSELoss,
    lr=0.1,
    **niceties,
)

In [92]:
from scipy.stats import loguniform, uniform

params = {
    "module__activation": ["relu", "elu", "softsign", "leaky_relu", "rrelu"],
    "batch_size": [32, 64, 128, 256],
    "optimizer__lr": loguniform(1e-4, 1e-1),
    "optimizer__weight_decay": loguniform(1e-6, 1e-3),
    "optimizer__momentum": uniform(0, 1),
    "optimizer__nesterov": [True],
}

In [93]:
search = HyperbandSearchCV(model, params, random_state=42, 
                           verbose=True, max_iter=9)

In [94]:
res = search.fit(X_train, y_train)

[CV, bracket=2] creating 9 models
[CV, bracket=1] creating 5 models
[CV, bracket=0] creating 3 models
[CV, bracket=0] For training there are between 71 and 71 examples in each chunk
[CV, bracket=1] For training there are between 71 and 71 examples in each chunk
[CV, bracket=0] validation score of 0.3760 received after 1 partial_fit calls
[CV, bracket=2] For training there are between 71 and 71 examples in each chunk
[CV, bracket=1] validation score of 0.2560 received after 1 partial_fit calls
[CV, bracket=2] validation score of 0.2640 received after 1 partial_fit calls
[CV, bracket=2] validation score of 0.3440 received after 3 partial_fit calls
[CV, bracket=1] validation score of 0.4080 received after 3 partial_fit calls
[CV, bracket=1] validation score of 0.2960 received after 9 partial_fit calls
[CV, bracket=2] validation score of 0.3440 received after 9 partial_fit calls
[CV, bracket=0] validation score of 0.4800 received after 9 partial_fit calls


In [65]:
res = search.fit(X_train, y_train)

[CV, bracket=2] creating 9 models


/home/sheer/miniconda3/envs/coiled-sheer-coiled-def_pytorch/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 5 is smaller than n_iter=9. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV, bracket=1] creating 5 models
[CV, bracket=0] creating 3 models
[CV, bracket=0] For training there are between 71 and 71 examples in each chunk
[CV, bracket=1] For training there are between 71 and 71 examples in each chunk
[CV, bracket=2] For training there are between 71 and 71 examples in each chunk
[CV, bracket=0] validation score of 0.3280 received after 1 partial_fit calls
[CV, bracket=1] validation score of 0.3920 received after 1 partial_fit calls
[CV, bracket=2] validation score of 0.3600 received after 1 partial_fit calls
[CV, bracket=1] validation score of 0.4480 received after 3 partial_fit calls
[CV, bracket=2] validation score of 0.4320 received after 3 partial_fit calls
[CV, bracket=0] validation score of 0.3440 received after 9 partial_fit calls
[CV, bracket=1] validation score of 0.3600 received after 9 partial_fit calls
[CV, bracket=2] validation score of 0.3600 received after 9 partial_fit calls


In [66]:
search.best_score_

0.36

In [95]:
search.best_score_

0.48

In [96]:
search.best_params_

{'batch_size': 128,
 'module__activation': 'leaky_relu',
 'optimizer__lr': 0.07114476009343418,
 'optimizer__momentum': 0.7319939418114051,
 'optimizer__nesterov': True,
 'optimizer__weight_decay': 6.251373574521747e-05}

In [97]:
search.best_estimator_

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=HiddenLayerNet(
    (fc1): Linear(in_features=768, out_features=100, bias=True)
    (extra_layers): ModuleList(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Linear(in_features=100, out_features=100, bias=True)
      (2): Linear(in_features=100, out_features=100, bias=True)
    )
    (fc2): Linear(in_features=100, out_features=5, bias=True)
  ),
)

In [101]:
from dask_ml.wrappers import ParallelPostFit
deployed_model = ParallelPostFit(search.best_estimator_)
deployed_model.score(X_test, y_test)

KeyboardInterrupt: 

In [69]:
from dask_ml.wrappers import ParallelPostFit
deployed_model = ParallelPostFit(search.best_estimator_)
deployed_model.score(X_test, y_test)

0.361

In [102]:
deployed_model.predict(X_test)

,Array,Chunk
Bytes,8.00 kB,64 B
Shape,"(1000,)","(8,)"
Count,250 Tasks,125 Chunks
Type,int64,numpy.ndarray


In [103]:
y_test.compute()

array([3, 2, 1, 2, 1, 1, 2, 4, 2, 3, 3, 2, 4, 4, 4, 3, 2, 1, 3, 4, 0, 2,
       3, 3, 4, 0, 4, 3, 0, 1, 4, 1, 1, 2, 4, 0, 2, 3, 1, 2, 0, 4, 0, 1,
       0, 2, 2, 1, 1, 3, 4, 1, 2, 1, 2, 3, 3, 0, 3, 3, 4, 1, 0, 0, 3, 1,
       3, 4, 3, 0, 4, 3, 4, 3, 2, 0, 0, 3, 1, 1, 2, 3, 2, 4, 0, 0, 2, 0,
       1, 3, 2, 4, 0, 1, 4, 2, 1, 0, 3, 4, 0, 1, 2, 2, 2, 4, 0, 3, 2, 4,
       2, 2, 1, 4, 2, 2, 1, 0, 0, 3, 0, 2, 4, 1, 1, 2, 3, 0, 1, 0, 2, 4,
       1, 3, 1, 0, 4, 1, 4, 1, 1, 3, 3, 3, 4, 3, 3, 2, 1, 1, 4, 0, 1, 2,
       3, 1, 4, 1, 1, 1, 4, 0, 1, 3, 1, 1, 2, 3, 4, 1, 1, 1, 3, 1, 2, 0,
       1, 0, 2, 4, 0, 1, 3, 4, 0, 3, 3, 0, 0, 0, 2, 4, 2, 2, 1, 4, 4, 0,
       2, 3, 0, 1, 4, 0, 1, 1, 0, 2, 3, 3, 1, 4, 3, 1, 3, 0, 2, 2, 3, 3,
       4, 3, 1, 2, 2, 0, 3, 1, 3, 2, 2, 1, 4, 0, 2, 4, 0, 4, 4, 4, 2, 1,
       0, 3, 1, 1, 0, 4, 3, 2, 1, 4, 4, 3, 0, 0, 2, 1, 2, 1, 3, 3, 2, 3,
       1, 1, 3, 0, 0, 4, 1, 0, 3, 3, 3, 3, 3, 4, 2, 4, 0, 3, 1, 2, 3, 2,
       1, 4, 2, 0, 0, 4, 0, 1, 4, 4, 3, 1, 4, 1, 4,

In [104]:
confusion_matrix(deployed_model.predict(X_test), y_test.compute())

KeyboardInterrupt: 

In [74]:
confusion_matrix(deployed_model.predict(X_test), y_test.compute())

array([[ 55,  30,  15,  11,  15],
       [ 55,  75,  45,  20,   7],
       [ 49,  66,  64,  33,  24],
       [ 13,  26,  34,  59,  43],
       [  9,  19,  49,  76, 108]])